<h1>IDEA !</h1>

Use **Python communication scripts** to act as a communication channel between **Unity C# scripts** and **Python controlling scripts**.

Use C# `Process` class to run Python communication scripts from the shell on the **C# side** and use the `os.system` method in python to run Python communciation scripts from shell on the **python side**

<h1>___put in a picture here___</h1>

<h3>Typical Python Daemon</h3>

In [57]:
%%file typical_daemon_file.py
#!/Library/Frameworks/Python.framework/Versions/3.4/bin/python3

import os,sys
import atexit
import signal

def daemonize(pidfile,stdin='/dev/null',stdout='/dev/null',stderr='/dev/null') : 
    if os.path.exists(pidfile) : 
        raise RuntimeError('Already running')
    
    # first fork (detach the child process from the parent 
    # process, and then terminate the parent)
    try : 
        if os.fork() > 0 : 
            raise SystemExit(0)  # parent exit
    except OSError as e : 
        raise RuntimeError('fork number 1 failed')
    else : 
        pass  # fork 1 success
    
    # it is a good practice to change the directory so that
    # the daemon is no longer working in the directory it was
    # launched from
    os.chdir('/')
    os.umask(0)
    
    os.setsid()  # the child process is now the session leader
    
    # second fork (relinquish session leadership)
    try : 
        if os.fork() > 0 : 
            raise SystemExit(0)
    except OSError as e : 
        raise RuntimeError('fork number 2 failed')
    else : 
        pass  # fork 2 success
        
    # replace the file descriptors for stdin, stdout and stderr
    with open(stdin,'rb',0) as f : 
        # dup2 duplicates the file descriptor, closing the latter
        # first if necessary
        os.dup2(f.fileno(),sys.stdin.fileno())
    with open (stdout,'ab',0) as f : 
        os.dup2(f.fileno(),sys.stdout.fileno())
    with open(stderr,'ab',0) as f : 
        os.dup2(f.fileno(),sys.stderr.fileno())
        
    # write the PID file
    with open(pidfile,'w') as f : 
        # write the PID of this daemon in the pid file (which in
        # this case is /tmp/daemon.pid)
        f.write(str(os.getpid()))
        
    # remove the PID file at exit
    atexit.register(lambda: os.remove(pidfile))
    
    # signal handler for termination
    def sigterm_handler(signo,frame) : 
        raise SystemExit(1)
    # this method is called when any signal is
    # passed to this process
        
    # signal.signal(signalnum, handler)
    # Set the handler for signal signalnum to the function handler. 
    # Handler can be a callable Python object
    signal.signal(signal.SIGTERM,sigterm_handler)
    
def main() : 
    import time
    sys.stdout.write('Daemon started with pid {}\n'.format(os.getpid()))
    while True : 
        sys.stdout.write('Daemon Alive\n')
        time.sleep(1)
    
if __name__ == '__main__' :  # this is the file which is run
    PIDFILE = '/tmp/daemon.pid'
    
    if len(sys.argv) != 2 : 
        error_text = 'Usage: {} [start|stop]'.format(sys.argv[0])
        print(error_text)
        # with open(sys.stderr,'wt') as f : 
        #     f.write(error_text)
        sys.stderr.write(error_text)
        raise SystemExit(1)
    
    if sys.argv[1] == 'start' :
        try : 
            daemonize(PIDFILE, stdout='/tmp/daemon.log', stderr='/tmp/daemon.log')
        except RuntimeError as e : 
            error_text = e
            print(e)
            sys.stderr.write(error_text)
            raise SystemExit(1)
        else : 
            pass # daemon started
        
        main()
        
    elif sys.argv[1] == 'stop' : 
        if os.path.exists(PIDFILE) :  # check if daemon is already running
            with open(PIDFILE) as f : 
                # os.kill(pid, sig) Send signal sig to the process pid. 
                # Constants for the specific signals available on the host 
                # platform are defined in the signal module.
                os.kill(int(f.read()),signal.SIGTERM)
                # in this case the file f (/tmp/daemon.pid) which stores the 
                # PID of the daemon process is read (returning the PID of the
                # daemon process). This PID is the first argument to the 
                # os.kill method, which means that this signal will be sent
                # to the daemonized process
                # -----------------------------------------------------------
                # we send the signal "signal.SIGTERM" to the daemon process.
                # since the "signal.signal(signal.SIGTERM,sigterm_handler)"
                # handler is defined for the signal "signal.SIGTERM", the 
                # sigterm_handler method is called, which is defined in
                # daemonize.py
        
        else : 
            error_text = 'daemon not running'
            print(error_text)
            sys.stderr.write(error_text)
            raise(SystemExit(1))
            
    else : 
        error_text = 'unknown command {}'.format(sys.argv[1])
        print(error_text)
        sys.stderr.write(error_text)
        raise SystemExit(1) 


Overwriting typical_daemon_file.py


In [58]:
import time
!rm /tmp/daemon.pid
!rm /tmp/daemon.log
!chmod +x typical_daemon_file.py
!./typical_daemon_file.py start
time.sleep(5)
!./typical_daemon_file.py stop

rm: /tmp/daemon.pid: No such file or directory


In [59]:
!cat /tmp/daemon.log

Daemon started with pid 95355
Daemon Alive
Daemon Alive
Daemon Alive
Daemon Alive
Daemon Alive
Daemon Alive


The daemon works perfectly, working for 5 seconds and then stopping, meanwhile writing the string "Daemon Alive" to /tmp/daemon.log every 1 second

We will now set up our text buffers, and modify the **typical_daemon_file.py** to write : 
 - python_read_daemon
 - python_write_daemon
 - unity_read_daemon
 - unity_write_daemon

In [60]:
%%file python_read_text_buffer

pass

Overwriting python_read_text_buffer


In [61]:
%%file unity_read_text_buffer

pass

Overwriting unity_read_text_buffer


We now write the daemonize function separately in a file **daemon.py** which we will import whenever we are defining an new type of daemon

In [78]:
%%file daemon.py
#!/Library/Frameworks/Python.framework/Versions/3.4/bin/python3

import os,sys
import atexit
import signal

def daemonize(pidfile,stdin='/dev/null',stdout='/dev/null',stderr='/dev/null') : 
    if os.path.exists(pidfile) : 
        raise RuntimeError('Already running')
    
    # first fork (detach the child process from the parent 
    # process, and then terminate the parent)
    try : 
        if os.fork() > 0 : 
            raise SystemExit(0)  # parent exit
    except OSError as e : 
        raise RuntimeError('fork number 1 failed')
    else : 
        pass  # fork 1 success
    
    # it is a good practice to change the directory so that
    # the daemon is no longer working in the directory it was
    # launched from
    os.chdir('/')
    os.umask(0)
    
    os.setsid()  # the child process is now the session leader
    
    # second fork (relinquish session leadership)
    # this step makes the daemon process give up the ability
    # to acquire a new controlling terminal (because only a 
    # session leader can do so) and provides even more isolation
    try : 
        if os.fork() > 0 : 
            raise SystemExit(0)
    except OSError as e : 
        raise RuntimeError('fork number 2 failed')
    else : 
        pass  # fork 2 success
        
    # replace the file descriptors for stdin, stdout and stderr
    with open(stdin,'rb',0) as f : 
        # dup2 duplicates the file descriptor, closing the latter
        # first if necessary.
        # syntax : dup(fd,fd2) where : 
        #         fd  --> file descriptor to be duplicated
        #         fd2 --> the duplicate file descriptor 
        os.dup2(f.fileno(),sys.stdin.fileno())
        # what we are doing here is that we are replacing 
        # sys.stdin with the file object returned by 
        # open(stdin,rb,0)
    with open (stdout,'ab',0) as f : 
        os.dup2(f.fileno(),sys.stdout.fileno())
    with open(stderr,'ab',0) as f : 
        os.dup2(f.fileno(),sys.stderr.fileno())
        
    # write the PID of the daemon in the pid file (which in this
    # case is "/tmp/daemon.pid")
    with open(pidfile,'w') as f : 
        f.write(str(os.getpid()))
        
    # remove the PID file at exit
    atexit.register(lambda: os.remove(pidfile))
    # since we need to supply a function to the "atexit.register" 
    # method, we define an inline function using the "lambda" 
    # keyword
    
    # signal handler for termination
    def sigterm_handler(signo,frame) : 
        raise SystemExit(1)
    # this method is called when any signal is
    # passed to this process
        
    # signal.signal(signalnum, handler)
    # Set the handler for signal signalnum to the function handler. 
    # Handler can be a callable Python object
    signal.signal(signal.SIGTERM,sigterm_handler)


Overwriting daemon.py


Now on to writing the daemon methods

In [114]:
%%file python_read_daemon.py
#!/Library/Frameworks/Python.framework/Versions/3.4/bin/python3

''' 
--------------------- specifications -----------------------------
 - start daemon process when called with the argument "start"
 - stop daemon process when called with the argument "stop"
 - read values from the text buffer when called with the argument "read"
------------------------------------------------------------------
'''

import sys,os
import signal
import daemon

def main() : 
    import time
    sys.stdout.write('Daemon started with pid {}\n'.format(os.getpid()))
    while True : 
        sys.stdout.write('Daemon Alive\n')
        time.sleep(1) 

if __name__ == '__main__' : 
    # is the top level module being called

    PIDFILE = '/tmp/python_read_daemon.pid'
    LOGFILE = '/tmp/python_read_daemon.log'

    def start() : 
        try : 
            daemon.daemonize(PIDFILE,stdout=LOGFILE,stderr=LOGFILE)
        except RuntimeError as e : 
            # daemon already runnning, as defined in the 
            # daemon.daemonize method
            print(e)
            sys.stderr.write(e)
            raise SystemExit(1)
        else : 
            # daemon started
            main()

    def stop() : 
        # check if daemon is already running
        if os.path.exists(PIDFILE) :
            # daemon is running
            with open(PIDFILE) as f : 
                os.kill(int(f.read()),signal.SIGTERM)
        else : 
            # daemon is not running
            error_text = 'Daemon is not running'
            print(error_text)
            sys.stderr.write(error_text)
            raise SystemExit(1)
            
    def read() : 
        pass

    command_method_dictionary = {
        'start' : start,
        'stop' : stop,
        'read' : read,
    }

    if len(sys.argv) == 1 :
        error_string = 'No arguments provided. Usage: {} [start|stop]'.format(sys.argv[0])
        print(error_string)
        sys.stderr.write(error_string)
        raise SystemExit(1)

    else : 
        try : 
            command_method_dictionary[sys.argv[1]]()
        except KeyError : 
            error_string = 'Incorrect argument provided\n acceptable arguments are :\n' 
            for command in command_method_dictionary.keys() : 
                error_string += command + '\n'
                
            print(error_string,file=sys.stderr)
#             sys.stderr.write(error_string)
            raise SystemExit(1)

Overwriting python_read_daemon.py


In [115]:
!rm /tmp/python_read_daemon.log
import time
!./python_read_daemon.py start
time.sleep(2)
!./python_read_daemon.py hmmm
time.sleep(1)
!./python_read_daemon.py stop
print('-------------------------------------')
!cat /tmp/python_read_daemon.log

rm: /tmp/python_read_daemon.log: No such file or directory
Incorrect argument provided
 acceptable arguments are :
stop
start
read

-------------------------------------
Daemon started with pid 2882
Daemon Alive
Daemon Alive
Daemon Alive
Daemon Alive


Now all that is remaining is to write the `read` method. Lets first decide what will be the format of the data that will be written in the **python_read_text_buffer**. Lets keep that for the next tutorial